# 构建语音识别系统 - 模型训练

## 构建模型

本次使用拟采用CTC的结构

CTC的结构如下图所示：

<img src="pic/CTCModel.png" alt="CTC Model" width="400" />

wavfrontend：进一步提取语音特征，做一下数据集的下采样

encoder：对语音特征进行编码，得到编码后的特征向量

softmax：将编码的特征向量通过线性层映射到词典大小，类似分类任务，预测出概率最大的词

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Tuple, Union
from model.subsampling import Subsampling
from model.encoder import LSTMEncoder
import math
from tokenizer.tokenizer import Tokenizer

class CTCModel(nn.Module):
    def __init__(self, in_dim, output_size,
                 vocab_size, blank_id
                ):
        super().__init__()
        self.subsampling = Subsampling(in_dim, output_size, subsampling_type=8)
        encoder_layer = nn.TransformerEncoderLayer(d_model=output_size, nhead=8)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)

        self.fc_out = nn.Linear(output_size, vocab_size)

        self.ctc_loss = torch.nn.CTCLoss(blank=blank_id,
                                         reduction="sum",
                                         zero_infinity=True)

    def forward(self, x, audio_lens, text, text_lens):
        x, encoder_out_lens = self.subsampling(x, audio_lens)
        x = self.encoder(x)
        predict = self.fc_out(x)

        predict = predict.transpose(0, 1)
        predict = predict.log_softmax(2)
        loss = self.ctc_loss(predict, text, encoder_out_lens, text_lens)
        loss = loss / predict.size(1)
        predict = predict.transpose(0, 1)
        return predict, loss, encoder_out_lens
    
    def inference(self, x, audio_lens):
        x, encoder_out_lens = self.subsampling(x, audio_lens)
        x = self.encoder(x)
        predict = self.fc_out(x)
        predict = predict.log_softmax(2)

        return predict, encoder_out_lens

tokenizer = Tokenizer("./tokenizer/vocab.txt")
model = CTCModel(80, 256, tokenizer.size(), tokenizer.blk_id())

d:\anaconda\envs\speechassistant\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


上面的模型是采用了 Transformer Encoder 作为声学模型，loss采用CTC loss。位于`model.model.CTCModel`。这里模型帮大家写好了。

这里简单说明一下CTC：在语音识别的时候，假设神经网络的输出长度为$L_{audio}$，识别文本经过tokenzier分词后，长度为$L_{text}$。一般情况下，$L_{audio} > L_{text}$

参考下面的代码

In [2]:
input = torch.load("./example1.pt")
audios = input["audios"]
audio_lens = input["audio_lens"]
texts = input["texts"]
text_lens = input["text_lens"]

pre, loss, pre_lens = model(audios, audio_lens, texts, text_lens)

In [3]:
print(pre_lens.view(-1).tolist())
print(text_lens.view(-1).tolist())

[64, 41, 75, 60, 56, 72, 19, 70, 59, 47, 41, 32, 80, 31, 26, 65]
[24, 14, 27, 21, 20, 22, 6, 27, 20, 17, 14, 12, 26, 12, 9, 25]


CTC的思路：

<img src="pic/CTC.png" alt="CTC" width="800" />

CTC引入了一个特殊字符，例如图中的$\epsilon$。对应我们tokenizer的special tokens里面的 \<blk\>。

In [4]:
print(tokenizer.special_token)

['<pad>', '<unk>', '<sos>', '<eos>', ' ', '<blk>']


CTC对输入的每一个时间步进行预测，根据对应的输出结果进行两步的处理：

- 将重复的字符压缩为一个
- 移除掉 $\epsilon$

例如上图中的一个解码路径： [$\epsilon$, a, $\epsilon$, $\epsilon$, b, $\epsilon$]

- 经过第一个步骤， [$\epsilon$, a, $\epsilon$, b, $\epsilon$]
- 经过第二个步骤，[a, b]

要得到[a,b]有许多情况，在训练的时候，目标是把这些路径的概率最大。

更加详细的参考[CTC](https://distill.pub/2017/ctc)。

## 训练

训练框架例如了 `run.py`里面的，大家可以根据自己的需求修改一下。

**TODO:** 把训练时候的loss记录下来，并作出曲线。